<a href="https://colab.research.google.com/github/shirakashi-jinnai/practice_tensorlflow/blob/main/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from sklearn.datasets import load_iris

iris = load_iris()
data = iris.data

print(data[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


In [24]:
import MeCab

str = '今日は晴れかな？'

# 天気の関連語をまとめたgazetteer
weather_set = set(['晴れ','天気','雨'])

chasen = MeCab.Tagger('-Ochasen')
wakati = MeCab.Tagger('-Owakati')
yomi = MeCab.Tagger('-Oyomi')
dump = MeCab.Tagger('-Odump')

tokens = chasen.parse(str)
token = tokens.split('\n')

for ele in token:
  element = ele.split('\t')
  # print(element)

  if element[0]=='EOS':
    break
  
  # 単語の表層を取得
  surface = element[0]

  if surface in weather_set:
    print(surface)



# print('chasen',chasen.parse(str))
# print('wakati',wakati.parse(str))
# print('yomi',yomi.parse(str))
# print('dump',dump.parse(str))

晴れ


In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



print(layers)

<module 'keras.api._v2.keras.layers' from '/usr/local/lib/python3.7/dist-packages/keras/api/_v2/keras/layers/__init__.py'>


In [ ]:
#TensorFlow を import して version を check　します
import tensorflow as tf
print('Tensorflow version:'+tf.__version__)

# tf.keras.datasets を使用して,ファッションMNISTデータセットをダウンロードすることから始めます
import numpy as np

test_data = tf.keras.datasets.fashion_mnist.load_data()



(x_train,y_train),(x_test,y_test)=tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)

# 次の例では3つのレイヤーを持つ標準の conv-net を使用し、各レイヤー間でドロップアウトとバッチ正規化を行います
def create_model():
  model=tf.keras.models.Sequential()

  model.add(tf.keras.layers.Conv2D(128,(3,3),input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=(3,3)))
  model.add(tf.keras.layers.Activation('elu'))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))

  return model

# トレーニングを開始するために TPU でモデルを作成してからコンパイルします
import os 

# TF 1.x
# resolver = tf.tpu.cont .cluster_resolver.TPUClusterResolver(tpu='grpc://'+os.environ['COLAB_TPU_ADDR'])
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy=tf.distribute.experimental.TPUStrategy(resolver)

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('Running on TPU',tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.experimental.TPUStrategy(tpu)
print('Replicas:',strategy.num_replicas_in_sync)

#
# Create model with TPU strategy
#
with strategy.scope():
  model = create_model()
  print('len!!!',model)
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      metrics=['accuracy'])

  model.fit(
      x_train.astype(np.float32),y_train.astype(np.float32),
      epochs=5,
      steps_per_epoch=60,
      validation_data=(x_test.astype(np.float32),y_test.astype(np.float32)),
      validation_freq=5
  )

  model.save_weights('./fashion_mnist.h5',overwrite=True)

  # 以上で TPU によるトレーニングが終了したので、モデルファッションカテゴリをどれだけ予測できるか見てみましょう
  LABEL_NAMES=['t_shirt','trouser','pullover','dress','coat','sandal','shirt','sneaker','bag','ankle_boots']

  cpu_model=create_model()
  cpu_model.load_weights('./fashion_mnist.h5')

  from matplotlib import pyplot
  %matplotlib inline

  def plot_predictions(images,predictions):
    n= images.shape[0]
    print('n=',n)
    nc = int(np.ceil(n/4))
    f,axes = pyplot.subplots(nc,4)
    for i in range(nc*4):
      y = i // 4
      x = i% 4
      axes[x,y].axis('off')
      print('range',np.argmax(predictions[i]))

      label = LABEL_NAMES[np.argmax(predictions[i])]
      confidence = np.max(predictions[i])

      if i > n:
        continue
      axes[x,y].imshow(images[i])
      axes[x,y].text(0.5,0.5,label+'\n%.3f'% confidence,fontsize=14)
    
    pyplot.gcf().set_size_inches(8,8)
  
  # print('→',x_test[:16],cpu_model.predict(x_test[:16]))

  plot_predictions(np.squeeze(x_test[:16]),cpu_model.predict(x_test[:16]))





Tensorflow version:2.8.2
Running on TPU ['10.69.160.26:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.69.160.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.69.160.26:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Replicas: 8


AttributeError: ignored

In [ ]:
#TensorFlow を import して version を check　します
import tensorflow as tf
print('Tensorflow version:'+tf.__version__)

# tf.keras.datasets を使用して,ファッションMNISTデータセットをダウンロードすることから始めます
import numpy as np

# データセットをロードし、サンプルを整数から浮動小数点に変換
test_data = tf.keras.datasets.fashion_mnist.load_data()
(x_train,y_train),(x_test,y_test)= tf.keras.datasets.mnist.load_data()
x_train,x_test=x_train/255.0,x_test/255.0

def model_creater():
  model = tf.keras.models.Sequential()
  layers=tf.keras.layers
  model.add(layers.Flatten(input_shape=(28,28)))
  model.add(layers.Dense(128,activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(10))
  return model

model = model_creater()

predictions = model(x_train[:1]).numpy()
print('predictions',predictions)

# クラスごとにこれらのロジットを確立に変換します
tf.nn.softmax(predictions).numpy()
print('end softmax')

# 損失関数は、ロジットと True の インデックスに関するベクトルを入力にとり、それぞれの標本についてクラスごとに損失のスカラーを返します。
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_fn(y_train[:1],predictions).numpy()

model.compile(optimizer='adam',loss=loss_fn,metrics=['accuracy'])
print('end model compile')

#Model.fitメソッドは損失を最小化するようにモデルのパラメータを調整します
model.fit(x_train,y_train,epochs=5)
print('end model fit')

# evaluteメソッドはモデルの性能を検査します。これには通常検査用データセットorテストデータセットを用います
model.evaluate(x_test,y_test,verbose=2)
print('end evaluate')

# モデルが確立を返すようにしたい場合は、モデルをラップしてソフトマックス関数を適用する
probability_model=tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(x_test[:5])

Tensorflow version:2.8.2
predictions [[-0.15955707  0.2144717   0.03854626  0.11279777  0.46448782  0.2224949
   0.545737   -0.00795412  0.04396196  0.04457785]]
end softmax
end model compile
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2930 - accuracy: 0.9138
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1398 - accuracy: 0.9588
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1057 - accuracy: 0.9672
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0865 - accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0743 - accuracy: 0.9765
end model fit
313/313 - 1s - loss: 0.0701 - accuracy: 0.9787 - 507ms/epoch - 2ms/step
end evaluate


<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.95839744e-08, 1.07796057e-08, 1.42541273e-06, 1.15028815e-04,
        2.84370097e-12, 1.22916873e-07, 1.54477702e-14, 9.99881625e-01,
        1.76682391e-07, 1.55427210e-06],
       [2.44364657e-07, 1.82538788e-04, 9.99807656e-01, 2.98489363e-06,
        5.86083335e-11, 2.12394752e-07, 2.62899050e-07, 8.28453178e-14,
        6.21016989e-06, 3.07750492e-11],
       [1.68830525e-07, 9.98688042e-01, 2.39906585e-04, 1.37580646e-05,
        1.77207563e-04, 1.48246829e-06, 9.90615081e-06, 8.23251961e-04,
        4.48821957e-05, 1.47888579e-06],
       [9.99683142e-01, 3.90369159e-08, 2.84762296e-04, 2.70855708e-07,
        1.08039046e-06, 1.87470403e-06, 2.66639727e-05, 9.28615123e-07,
        8.73195916e-08, 1.18453943e-06],
       [3.58038301e-06, 7.01997419e-11, 1.97412610e-05, 4.17848547e-08,
        9.97925282e-01, 1.03352500e-06, 2.16828039e-06, 8.24084782e-06,
        3.66562858e-06, 2.03627115e-03]], dtype=float32)>